# H2O ML Using Python 

# Running the H2OAutoML
## REMEMBER THE PROCESS MAY TAKE >5 MINUTES

In [ ]:
import pandas as pd
import numpy as np
import h2o

In [ ]:
#Starting h2o
h2o.init()

In [ ]:
#Read the training and test data
train = pd.read_csv("../input/tabular-playground-series-jan-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-jan-2021/test.csv")


In [ ]:
# convert pandas DataFrame into H2O Frame 
h2o_train = h2o.H2OFrame(train) 
# Describe  the train h20Frame 
h2o_train.describe() 


In [ ]:
#set the x & y(target) to fit into the algorithm
x = h2o_train.columns
y = 'target'
x.remove(y)

#### The process may take long, just grab a coffe and wait till the process ends

In [ ]:
# import autoML from H2O 
from h2o.automl import H2OAutoML 
# callh20automl  function  
aml = H2OAutoML(
    max_runtime_secs = 3000, 
    exclude_algos =['DeepLearning'], 
    seed = 42, 
    stopping_metric ='RMSE', 
    sort_metric ='RMSE', 
    # balance_classes = False,
    max_models=60
) 
# train model
aml.train(x = x, y = y, training_frame = h2o_train) 


In [ ]:
# # View the H2O aml leaderboard 
lb = aml.leaderboard 
# Print all rows instead of 10 rows 
lb.head(rows = lb.nrows)

In [ ]:
#convert test  to h2o dataframe
h2o_test = h2o.H2OFrame(test) 

In [ ]:
'''
- h2o.as_list will convert the predictor as float64 object, from h2o dataframe
- .values will get just the values from the object
'''
pred = h2o.as_list(aml.leader.predict(h2o_test)).values

In [ ]:
submissions = pd.read_csv("../input/tabular-playground-series-jan-2021/sample_submission.csv")
submissions.target = pred
submissions.to_csv("submission.csv",index=False)

In [ ]:
!kaggle competitions submit -c tabular-playground-series-jan-2021 -f submission.csv -m "Message"